In [1]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn

# Spark

In [2]:
! /home/jovyan/start-hadoop.sh

jovyan
 * Starting OpenBSD Secure Shell server sshd
start-stop-daemon: unable to set gid to 0 (Operation not permitted)
   ...fail!
 * sshd is running
Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: namenode is running as process 165.  Stop it first and ensure /tmp/hadoop-jovyan-namenode.pid file is empty before retry.
Starting datanodes
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: datanode is running as process 278.  Stop it first and ensure /tmp/hadoop-jovyan-datanode.pid file is empty before retry.
Starting secondary namenodes [e88e4034d828]
e88e4034d828: Warning: Permanently added 'e88e4034d828' (ED25519) to the list of known hosts.
e88e4034d828: secondarynamenode is running as process 489.  Stop it first and ensure /tmp/hadoop-jovyan-secondarynamenode.pid file is empty before retry.
Starting resourcemanager
resourcemanager is running as process

In [3]:
# connect, context, session

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-03-21 21:33:59,112 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## Outbrain click prediction dataseet

https://www.kaggle.com/c/outbrain-click-prediction/data - you need to register here

In [4]:
!pip install -q kaggle

In [ ]:
# https://github.com/Kaggle/kaggle-api
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# instruction where to get ~/.kaggle/kaggle.json file

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [5]:
! mkdir -p ~/.kaggle

In [6]:
%%writefile ~/.kaggle/kaggle.json
{"username":"veronikanikonova","key":"225b2f7497774e9f2ab634602a1e8cac"}

Writing /home/jovyan/.kaggle/kaggle.json


In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! pip install -U urllib3 kaggle==1.5.3
! kaggle competitions download -c outbrain-click-prediction -f page_views_sample.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f events.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_topics.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_entities.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f documents_meta.csv.zip 
! kaggle competitions download -c outbrain-click-prediction -f clicks_test.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f clicks_train.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f documents_categories.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f promoted_content.csv.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 526.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.3-py3-none-any.whl size=64965 sha256=1b785b75938fd5114e20ba1967b1f52452f8fb912dfa4ce114f542235b6aa14f
  Stored in directory: /home/jovyan/.cache/pip/wheels/ec/9c/e3/864c634d8176e91b4845d2a12f7ae6b1c035d56457f81452d0
Successfully built kaggle
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13
100%|████████████████████████████████████████| 149M/149M [00:40<00:00, 4.47MB/s]
100%|████████████████████████████████████████| 149M/149M [00:40<00:00, 3.86MB/s]
100%|█████

In [9]:
! unzip '*.zip'
! rm -rf *.zip

Archive:  documents_entities.csv.zip
  inflating: documents_entities.csv  

Archive:  page_views_sample.csv.zip
  inflating: page_views_sample.csv   

Archive:  clicks_train.csv.zip
  inflating: clicks_train.csv        

Archive:  events.csv.zip
  inflating: events.csv              

Archive:  documents_categories.csv.zip
  inflating: documents_categories.csv  

Archive:  clicks_test.csv.zip
  inflating: clicks_test.csv         

Archive:  documents_topics.csv.zip
  inflating: documents_topics.csv    

Archive:  promoted_content.csv.zip
  inflating: promoted_content.csv    

Archive:  documents_meta.csv.zip
  inflating: documents_meta.csv      

9 archives were successfully processed.


In [4]:
! hdfs dfs -put page_views_sample.csv
! hdfs dfs -put events.csv
! hdfs dfs -put documents_topics.csv
! hdfs dfs -put documents_entities.csv
! hdfs dfs -put documents_meta.csv
! hdfs dfs -put clicks_test.csv
! hdfs dfs -put clicks_train.csv
! hdfs dfs -put documents_categories.csv
! hdfs dfs -put promoted_content.csv

put: `page_views_sample.csv': File exists
put: `events.csv': File exists
put: `documents_topics.csv': File exists
put: `documents_entities.csv': File exists
put: `documents_meta.csv': File exists
put: `clicks_test.csv': File exists
put: `clicks_train.csv': File exists
put: `documents_categories.csv': File exists
put: `promoted_content.csv': File exists


In [10]:
# load data
df = se.read.csv("page_views_sample.csv", header=True)
df.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



In [12]:
from IPython.display import display
tables = ["clicks_test", "clicks_train", 
          "documents_categories", "documents_entities", "documents_meta", "documents_topics", 
          "events", "page_views_sample", "promoted_content"]
for name in tqdm.tqdm(tables):
    df = se.read.csv("{}.csv".format(name), header=True)
    df.registerTempTable(name)
    print(name)
    df.limit(3).show()

  0%|          | 0/9 [00:00<?, ?it/s]

clicks_test
+----------+------+
|display_id| ad_id|
+----------+------+
|  16874594| 66758|
|  16874594|150083|
|  16874594|162754|
+----------+------+

clicks_train
+----------+------+-------+
|display_id| ad_id|clicked|
+----------+------+-------+
|         1| 42337|      0|
|         1|139684|      0|
|         1|144739|      1|
+----------+------+-------+

documents_categories
+-----------+-----------+----------------+
|document_id|category_id|confidence_level|
+-----------+-----------+----------------+
|    1595802|       1611|            0.92|
|    1595802|       1610|            0.07|
|    1524246|       1807|            0.92|
+-----------+-----------+----------------+

documents_entities
+-----------+--------------------+-----------------+
|document_id|           entity_id| confidence_level|
+-----------+--------------------+-----------------+
|    1524246|f9eec25663db4cd83...|0.672865314504701|
|    1524246|55ebcfbdaff1d6f60...|0.399113728441297|
|    1524246|839907a972930b17b

## Evaluation Assignment

Data: outbrain click prediction

Tasks:
Using Spark RDD, DataFrame API and Python, calculate:

**1**. Top 10 most visited document_ids in the page_views_sample log

**2**. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

**3***. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

The submission format is the result.json json file with top_10_documents, users and top_10_topics keys.
For TOP-10 results, the answer must be written in the form of a sheet ordered from TOP-1 to TOP-10 with an id.

result.json example:

    {
        "top_10_documents": [
            111,
            222,
            333,
            ...,
            1010
        ],
        "users": 10000,
        "top_10_topics": [
            11,
            22,
            33,
            ...,
            101
        ]
    }

In [13]:
# Task 1. Variant 1 (using sql)
top_10_docs = se.sql("""
select
    document_id, 
    count(*) as cnt 
from page_views_sample 
group by document_id 
order by cnt desc
""")
top_10_docs.limit(10).show()

+-----------+------+
|document_id|   cnt|
+-----------+------+
|    1811567|429551|
|        234|179692|
|      42744|156231|
|    1858440|112140|
|    1780813|109624|
|      60164|104941|
|    1790442| 91420|
|    1877626| 80309|
|    1821895| 79956|
|     732651| 74630|
+-----------+------+



In [14]:
from pyspark.sql import functions as F
from pyspark.sql.functions import desc

# Task 1. Variant 2 (using DataFrame)
page_views = se.table("page_views_sample")
(
  page_views
    .groupby("document_id")
    .agg(F.count("document_id").alias("cnt"))
    .orderBy(desc("cnt"))
    .limit(10)
    .show()
)

+-----------+------+
|document_id|   cnt|
+-----------+------+
|    1811567|429551|
|        234|179692|
|      42744|156231|
|    1858440|112140|
|    1780813|109624|
|      60164|104941|
|    1790442| 91420|
|    1877626| 80309|
|    1821895| 79956|
|     732651| 74630|
+-----------+------+



In [17]:
# Task 2. Variant 1 (using sql)
users_diff_ts = se.sql("""
select
    count(distinct pvs_1.uuid) 
from page_views_sample as pvs_1 
cross join page_views_sample as pvs_2 
where pvs_1.uuid = pvs_2.uuid and pvs_1.traffic_source <> pvs_2.traffic_source
""")
users_diff_ts.show()

+--------------------+
|count(DISTINCT uuid)|
+--------------------+
|               98080|
+--------------------+



In [17]:
# Task 2. Variant 2 (using sql)
users_diff_ts = se.sql("""
select
    count(*) as users_number
from(
    select
        uuid,
        count(distinct traffic_source) as cnt
    from page_views_sample 
    group by uuid
    having cnt > 1
)
""")
users_diff_ts.show()

+------------+
|users_number|
+------------+
|       98080|
+------------+



In [27]:
# Task 2. Variant 3 (using DataFrame)
(
  page_views
    .groupby("uuid")
    .agg(F.countDistinct("traffic_source").alias("cnt"))
    .where("cnt > 1")
    .count()
)

98080

In [28]:
# Task 3. Variant 1 (using sql)
top_topics = se.sql("""
select
    dt.topic_id, 
    count(*) as cnt 
from page_views_sample as pvs_1 
join documents_topics as dt 
on pvs_1.document_id=dt.document_id 
group by dt.topic_id 
order by cnt desc
""")
top_topics.limit(10).show()

+--------+-------+
|topic_id|    cnt|
+--------+-------+
|      20|1429253|
|      16|1425838|
|     216|1160563|
|     136|1099382|
|     140| 971983|
|     143| 919136|
|      36| 855793|
|      97| 839328|
|       8| 819599|
|     269| 727145|
+--------+-------+



In [29]:
# Task 3. Variant 2 (using DataFrame)
documents_topics = se.table("documents_topics")
(
  page_views
    .join(documents_topics, page_views.document_id == documents_topics.document_id, 'inner')
    .groupby("topic_id")
    .agg(F.count("topic_id").alias("cnt"))
    .orderBy(desc("cnt"))
    .limit(10)
    .show()
)

+--------+-------+
|topic_id|    cnt|
+--------+-------+
|      20|1429253|
|      16|1425838|
|     216|1160563|
|     136|1099382|
|     140| 971983|
|     143| 919136|
|      36| 855793|
|      97| 839328|
|       8| 819599|
|     269| 727145|
+--------+-------+



In [73]:
import json

data = json.dumps({
    "top_10_documents": [
        1811567,
        234,
        42744,
        1858440,
        1780813,
        60164,
        1790442,
        1877626,
        1821895,
        732651        
    ],
    "users": 98080,
    "top_10_topics": [
        20,
        16,
        216,
        136,
        140,
        143,
        36,
        97, 
        8, 
        269
    ]
})
with open('result.json', 'w') as f:
    f.write(data)

In [74]:
!curl -F file=@result.json "51.250.54.133:80/MDS-LSML1/veronika_nikon/w4/1"

1.0
Well done!
